In [ ]:
import io
import zipfile

from glob import glob

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pydicom
from pydicom.filebase import DicomBytesIO

from pymedphys.mudensity import calc_mu_density, get_grid
from pymedphys.dcm import dcm_from_dict

In [ ]:
zip_filepath = glob('../../../tests/data/logfiles/Trilogy/*')[0]

In [ ]:
with open(zip_filepath, 'rb') as input_file:
    data = io.BytesIO(input_file.read())

In [ ]:
data_zip = zipfile.ZipFile(data)

In [ ]:
data_zip.namelist()

In [ ]:
namelist = data_zip.namelist()

dicom_files = [path for path in namelist if path.endswith('.dcm')]
dicom_files

In [ ]:
with data_zip.open('VMAT_TS4_2-field_H&N/RP.VMAT_TS4_2-field.dcm') as fp:
    raw = DicomBytesIO(fp.read())
    dcm_original = pydicom.dcmread(raw)

In [ ]:
leaf_widths = [5,5,5]

In [ ]:
# calc_mu_density?

In [ ]:
MU = [0, 10, 20]


MLC = np.array([
    [
        [3, -3],
        [3, -3],
        [3, -3]
    ],
    [
        [3, 3],
        [3, 3],
        [3, 3]
    ],
    [
        [-3, 3],
        [-3, 3],
        [-3, 3]
    ]
])

JAW = np.array([
    [3, 3],
    [3, 3],
    [3, 3]
])

LEAF_PAIR_WIDTHS = [2, 2, 2]

MAX_LEAF_GAP = 8

mu_density = calc_mu_density(MU, MLC, JAW, leaf_pair_widths=LEAF_PAIR_WIDTHS, max_leaf_gap=MAX_LEAF_GAP, grid_resolution=2)

mu_density

In [ ]:
get_grid(leaf_pair_widths=LEAF_PAIR_WIDTHS, max_leaf_gap=MAX_LEAF_GAP, grid_resolution=2)

In [ ]:
leaf_boundaries = [0] + np.cumsum(LEAF_PAIR_WIDTHS).tolist()
leaf_boundaries = np.array(leaf_boundaries) - np.mean(leaf_boundaries)

dcm = dcm_from_dict({
    'BeamSequence': [
        {
            'BeamLimitingDeviceSequence': [
                {
                    'RTBeamLimitingDeviceType': 'MLCX',
                    'LeafPositionBoundaries': leaf_boundaries,
                    'NumberOfLeafJawPairs': len(leaf_boundaries) - 1
                }
            ]
        }
    ]
})

In [ ]:
MU[0]

In [ ]:
dcm

In [ ]:
dcm_original

In [ ]:
# dcm

In [ ]:
beam_sequence = dcm.BeamSequence[0]

In [ ]:
mlc_beam_limiting_device = beam_sequence.BeamLimitingDeviceSequence[2]

In [ ]:
leaf_boundaries = mlc_beam_limiting_device.LeafPositionBoundaries
leaf_widths = np.diff(leaf_boundaries)
leaf_widths

In [ ]:
beam_sequence.SourceAxisDistance

In [ ]:
mlc_beam_limiting_device

In [ ]:
mlc_pos_scaling_factor = beam_sequence.SourceAxisDistance / mlc_beam_limiting_device.SourceToBeamLimitingDeviceDistance
mlc_pos_scaling_factor

In [ ]:
# https://github.com/jrkerns/pylinac/blob/master/pylinac/log_analyzer.py#L781
pylinac_dynalog_leaf_conversion = 1.96614

In [ ]:
assert beam_sequence.BeamLimitingDeviceSequence[2].NumberOfLeafJawPairs == len(leaf_widths)

num_leaves = len(leaf_widths)

In [ ]:
control_points = beam_sequence.ControlPointSequence

In [ ]:
dcm_mlcs = [
    control_point.BeamLimitingDevicePositionSequence[-1].LeafJawPositions
    for control_point in control_points
]

dcm_mlcs = [
    np.array([mlc[num_leaves::], -np.array(mlc[0:num_leaves])]).T
    for mlc in dcm_mlcs
]

dcm_mlcs = np.array(dcm_mlcs)
np.shape(dcm_mlcs)

In [ ]:
control_points[0].BeamLimitingDevicePositionSequence[1]

In [ ]:
dicom_jaw = control_points[0].BeamLimitingDevicePositionSequence[1].LeafJawPositions

dcm_jaw = np.array(dicom_jaw)[-1::-1]
dcm_jaw[1] = -dcm_jaw[1]

dcm_jaw = dcm_jaw[None,:] + np.zeros([50,1])

np.shape(dcm_jaw)

In [ ]:
total_mu = np.array(dcm.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
total_mu

In [ ]:
final_mu_weight = np.array(beam_sequence.FinalCumulativeMetersetWeight)
final_mu_weight

In [ ]:
dcm_mu = [
    total_mu * np.array(control_point.CumulativeMetersetWeight) / final_mu_weight
    for control_point in control_points
]
dcm_mu = np.array(dcm_mu)
np.shape(dcm_mu)

In [ ]:
dcm_mu_density = calc_mu_density(dcm_mu, dcm_mlcs, dcm_jaw, leaf_pair_widths=leaf_widths, grid_resolution=5)

In [ ]:
# pd.read_csv?

In [ ]:
with data_zip.open('VMAT_TS4_2-field_H&N/A20190110141052_Anonymous.dlg') as fp:
    dfA = pd.read_csv(fp, skiprows=6, header=None)
    
with data_zip.open('VMAT_TS4_2-field_H&N/B20190110141052_Anonymous.dlg') as fp:
    dfB = pd.read_csv(fp, skiprows=6, header=None)

In [ ]:
# dfA

In [ ]:
dynalog_arrayA = dfA.values
dynalog_arrayB = dfB.values
# dynalog_arrayA[-1,:]

In [ ]:
Y1A = dynalog_arrayA[:,8]
Y1B = dynalog_arrayB[:,8]

assert np.all(Y1A == Y1B)

Y2A = dynalog_arrayA[:,9]
Y2B = dynalog_arrayB[:,9]

assert np.all(Y2A == Y2B)


jaw = np.array([Y1A, Y2A]).T
np.shape(jaw)

# jaw

In [ ]:
# mlc_1 = dynalog_array[:,14::4]
mlc_A = dynalog_arrayA[:,15::4] / 100
mlc_B = dynalog_arrayB[:,15::4] / 100
# mlc_3 = dynalog_array[:,16::4]
# mlc_4 = dynalog_array[:,17::4]

## Couldn't find total MU with Logfile

In [ ]:
### COULDN'T FIND TOTAL MU WITHIN LOGFILE ###
assumed_total_mu = total_mu

In [ ]:
muA = dynalog_arrayA[:,0] / np.max(dynalog_arrayA[:,0]) * assumed_total_mu
muB = dynalog_arrayB[:,0] / np.max(dynalog_arrayB[:,0]) * assumed_total_mu

assert np.all(muA == muB)

mu = muA

np.shape(mu)

In [ ]:
# mlc_1

In [ ]:
mlcs = np.swapaxes(np.array([mlc_A, mlc_B]).T, 0, 1)
mlcs = mlcs

mlcs = pylinac_dynalog_leaf_conversion * mlcs
np.shape(mlcs)

In [ ]:
# for i in range(60):
#     plt.figure()
#     plt.plot(mlc_A[:,i])
#     plt.plot(-mlc_B[:,i])

#     plt.title(i)
#     plt.show()

In [ ]:
# calc_mu_density?

In [ ]:
mu_density = calc_mu_density(mu, mlcs, jaw, leaf_pair_widths=leaf_widths, grid_resolution=5)

In [ ]:
grid = get_grid(leaf_pair_widths=leaf_widths,grid_resolution=5)

In [ ]:
plt.figure(figsize=(10,8.5))
plt.pcolormesh(grid['mlc'][-1::-1], grid['jaw'][-1::-1], mu_density)
plt.colorbar()
plt.xlabel('MLCX direction (mm)')
plt.ylabel('ASYMY travel direction (mm)')

plt.axis('equal')

In [ ]:
plt.figure(figsize=(10,8.5))
plt.pcolormesh(grid['mlc'][-1::-1], grid['jaw'][-1::-1], dcm_mu_density)
plt.colorbar()
plt.xlabel('MLCX direction (mm)')
plt.ylabel('ASYMY travel direction (mm)')

plt.axis('equal')

In [ ]:
np.shape(dcm_mu_density)

In [ ]:
diff = mu_density - dcm_mu_density
max_range = np.max(np.abs(diff))


plt.figure(figsize=(10,8.5))
plt.pcolormesh(grid['mlc'][-1::-1], grid['jaw'][-1::-1], diff, vmin=-max_range, vmax=max_range, cmap='bwr')
plt.colorbar()
plt.xlabel('MLCX direction (mm)')
plt.ylabel('ASYMY travel direction (mm)')

plt.axis('equal')

In [ ]:
dicom_colour = '#1f77b4'
logfile_colour = '#ff7f0e'


for i in range(60):
    plt.figure()
    plt.title("Leaf #{}".format(i+1))
    plt.plot(dcm_mu, dcm_mlcs[:,i,0], c=dicom_colour, label='dicom')
    plt.plot(mu, mlcs[:,i,0], c=logfile_colour, label='logfile')
    
    plt.legend()
    
    plt.plot(dcm_mu, -dcm_mlcs[:,i,1], c=dicom_colour)
    plt.plot(mu, -mlcs[:,i,1], c=logfile_colour)
    plt.show()